# Transcripts per reaction

By Garrett Roell and Christina Schenk

Tested on biodesign_3.7 kernel on jprime

This notebook determines the number of transcripts per reaction for a given transc

### Method: 
<ol>
<li>Setup imports</li>
<li>Load model and relevant data</li>
<li>Isolate wildtype phenol and glucose parameters</li>
    
<li>Run phenol FBA with many levels of ATP maintenance flux</li>
<li>Plot phenol growth rate vs. ATP maintenance flux</li>
<li>Run FVA to calculate phenol ATP maintenance flux value</li>
    
<li>Run glucose FBA with many levels of ATP maintenance flux</li>
<li>Plot glucose growth rate vs. ATP maintenance flux</li>
<li>Run FVA to calculate glucose ATP maintenance flux value</li>
</ol>

### 1. Setup imports

In [ ]:
import cobra

import sys
source_dir = '../src'
sys.path.append(source_dir)

from t_to_f_functions import *

### 2. Load model and relevant data

In [ ]:
model = cobra.io.read_sbml_model("../models/r_opacus_annotated_curated.xml")

# load transcript data
transcript_df = pd.read_csv('../data/transcript_data/henson_CPM_melted.csv')

# isolate wt glucose data
glucose_transcript_df = transcript_df[transcript_df['Line Name'].str.contains("WT-Glu")]
glucose_transcript_df = glucose_transcript_df[glucose_transcript_df['Time'] == 10]

# isolate wt phenol data
phenol_wt_transcript_df = transcript_df[transcript_df['Line Name'].str.contains("WT-P")]

# isolate pvhg6 phenol data
phenol_pvhg6_transcript_df = transcript_df[transcript_df['Line Name'].str.contains("PVHG-P")]

### 3. 

In [8]:
# for trial in ['WT-Glu-R1', 'WT-Glu-R2','WT-Glu-R3']:
trial = 'WT-Glu-R1'
transcript_df = transcript_df_T1[transcript_df_T1['Line Name'] == trial]

# remove unneeded columns
transcript_df = transcript_df.filter(['Count', 'Measurement Type'])

# set index of dataframe to be gene names
transcript_df = transcript_df.set_index('Measurement Type')

transcript_df.head()

,Count
Measurement Type,
WP_005263480_1,27.446117
WP_005249107_1,355.352526
WP_005249106_1,131.395949
WP_005249105_1,64.507709
WP_005249104_1,42.802872


In [13]:
eflux2_model = model.copy()
eflux2_model.tolerance = 1e-9

transcipt_bounds = []

# set the flux bounds for each reaction using the transcriptomics data    
for rxn in eflux2_model.reactions:
    if 'EX_' not in str(rxn.id):
        if rxn.gene_reaction_rule:
            if rxn.lower_bound < 0.0:
                rxn.lower_bound = -transcript_value_for_rxn(model, transcript_df, rxn)
            else:
                pass
            if rxn.upper_bound > 0.0:
                rxn.upper_bound = transcript_value_for_rxn(model, transcript_df, rxn)
            else:
                pass


        else:
            """When there is no GPR, the arbitrary bounds are removed. 
            Common arbitrary bound value of 1000 for E.coli, might be different depending on the model, e.g., 99999.0 for iMM904 yeast model in BiGG"""
            if rxn.lower_bound <= -1000:
                rxn.lower_bound = -np.Inf
            if rxn.upper_bound >= 1000:
                rxn.upper_bound = np.Inf 
                
    transcipt_bounds.append({
        "reaction": rxn.id,
        "lower_bound": rxn.lower_bound,
        "upper_bound": rxn.upper_bound
    })
#     print(rxn.id, rxn.lower_bound, rxn.upper_bound)

In [15]:
transcript_bounds_df = pd.DataFrame(transcipt_bounds)
transcript_bounds_df.head()

,reaction,lower_bound,upper_bound
0,12DGR140tipp,0.000000,inf
1,13PPDH,-38.415228,38.415228
2,1P2CBXLCYCL,-inf,inf
3,1P2CBXLR,0.000000,0.840187
4,23CTI1,0.000000,23.058472


In [17]:
trial_name = 'glucose_transcripts_per_reaction_1'
transcript_bounds_df.to_csv(f'../data/transcript_data/{trial_name}.csv', index=False)

In [18]:
model.reactions.get_by_id('CYANST')

Reaction identifier,CYANST
Name,Cyanide sulfurtransferase
Memory address,0x07fad5c8a9450
Stoichiometry,cyan_c + tsul_c --> h_c + so3_c + tcynt_c Hydrogen cyanide + Thiosulfate --> H+ + Sulfite + Thiocyanate
GPR,WP_005238137_1 or WP_005239980_1 or WP_005244352_1 or WP_005251647_1
Lower bound,0.0
Upper bound,1000.0
